<font size=3>

<font size=4> **Örnek 1** </font>

A ürününün bir asortisi 6 parça, B ürününün bir asortisiyse 5 parça ürün içeriyor. Sevkiyat kısıtı gereği bir seferde 60 taneden fazla ürün mağazaya gönderilemiyor. Magaza A ürününün bir asortisi 10 birim alan, B ürününün bir asortisiyse 20 birim alan kaplıyor. Mağazada bu ürünlerin toplam sergileme alanıysa 150 birim. Bunların yanı sıra A ürününden depoda toplam 6 asorti var. A ürününün bir asortisi 5, B ürününün bir asortisiyse 4.5 TL kar getiriyorsa, en yüksek karı veren karışımı bulunuz. 

In [1]:
from pulp import *
# import pulp
# from pulp import LpVariable

In [2]:
# Initialize Class
model = LpProblem("XYZ Kar En İyileştirme", LpMaximize) #LpMinimize

# Define Decision Variables
X = LpVariable('A', lowBound=0, upBound=None, cat='Integer')
Y = LpVariable('B', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 5 * X + 4.5 * Y

# Define Constraints
model += 6 * X + 5 * Y <= 60
model += 10 * X + 20 * Y <= 150
model += X <= 6

# Solve Model
model.solve()

1

In [3]:
#Solution
print("Send {} assortment of product A".format(X.varValue))
print("Send {} assortment of product A".format(Y.varValue))

Send 6.0 assortment of product A
Send 4.0 assortment of product A


<font size=4> **Örnek 2** </font>

<font size=3>**2.1. LpSum kullanımı ve dictionary**</font>

Lojistik Planlama problemi.

Önümüzdeki ayın lojistik planlaması için iki depodan (İstanbul, konya) 4 farklı mağaza (T100, T101, T102, T103) yapılabilecek en uygun planı oluşturun. T100 mağazasının toplam talebi 1800, T101'in 1200, T102'nin 1100 ve T103'ün 1000. Depolardan mağazalara nakliyat masrafı aşağıdaki gibidir:<br>
             İstanbul   Konya<br>
T101      211            232<br>
T102      232            212<br>
T103      240            230<br>
T104      300            280<br>

<font size=3> 2.1.1. Dictinoary Veri Yapısı</font>

In [4]:
#Single Index
d = {'T100': 1800, 'T101':1200, 'T102':1100, 'T103':1000}
display(d['T103'])

#Double Index
x = {('K','0'):'XK0', ('K','1'):'XK1', ('K','2'):'XK2', ('K','3'):'XK3', ('I','0'):'XI0', ('I','1'):'XI1', ('I','2'):'XI2', ('I','3'):'XI3'}
display(x[('K','0')])

1000

'XK0'

<font size=3> 2.1.2. Model</font>

In [5]:
XK0 = LpVariable('XK0', lowBound=0, cat='Integer')
XK1 = LpVariable('XK1', lowBound=0, cat='Integer')
XK2 = LpVariable('XK2', lowBound=0, cat='Integer')
XK3 = LpVariable('XK3', lowBound=0, cat='Integer')
XI0 = LpVariable('XI0', lowBound=0, cat='Integer')
XI1 = LpVariable('XI1', lowBound=0, cat='Integer')
XI2 = LpVariable('XI2', lowBound=0, cat='Integer')
XI3 = LpVariable('XI3', lowBound=0, cat='Integer')

costs = {('Konya', 'T100'): 232, 
            ('Konya', 'T101'): 230,
            ('Konya', 'T102'): 212,
            ('Konya', 'T103'): 280,
            ('Istanbul', 'T100'): 211, 
            ('Istanbul', 'T101'): 240,
            ('Istanbul', 'T102'): 232,
            ('Istanbul', 'T103'): 300}

x = {('Konya', 'T100'): XK0, 
            ('Konya', 'T101'): XK1,
            ('Konya', 'T102'): XK2,
            ('Konya', 'T103'): XK3,
            ('Istanbul', 'T100'): XI0, 	
            ('Istanbul', 'T101'): XI1,
            ('Istanbul', 'T102'): XI2,
            ('Istanbul', 'T103'): XI3}

<font size=3> 2.1.3. List Comprehension</font>

In [6]:
#ex1
display([i**2 for i in [1,2,3,4,5]])

#ex2
display([d[i] for i in ['T100', 'T101', 'T102', 'T103']])

[1, 4, 9, 16, 25]

[1800, 1200, 1100, 1000]

In [7]:
# Initialize Model
model2 = LpProblem("Minimize Transportation Costs", LpMinimize)

# Build the lists and the demand dictionary
# ranges/index
warehouse = ['Istanbul', 'Konya']
stores = ['T100', 'T101', 'T102', 'T103']
store_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(stores, store_demand))

# Define Objective
model2 += lpSum([costs[(w, s)] * x[(w, s)] for s in stores for w in warehouse])

# For each customer, sum warehouse shipments and set equal to customer demand
for s in stores:
    model2 += lpSum([x[(w, s)] for w in warehouse]) == demand[s]
    
# Solve Model
model2.solve()

1

In [27]:
#Q1 How to print all of the solutions?
print("Send from Istanbul to T100 {} of products".format(XI0.varValue))

#Optimal Total Cost
print("Optimal Total Cost {} TL".format(value(model.objective)))

Send from Istanbul to T100 1800.0 of products
Optimal Total Cost 1079800.0 TL


<font size=3>**2.2. LpVariable.dicts kullanımı ve model çıktılar**</font>

In [8]:
# Initialize Model
model3 = LpProblem("Minimize Transportation Costs", LpMinimize)

# ranges/index
warehouse = ['Istanbul', 'Konya']
stores = ['T100', 'T101', 'T102', 'T103']
store_demand = [1800, 1200, 1100, 1000]

#parameters
demand = dict(zip(stores, store_demand))

# Define decision variables
#Alternative1
index = [(w, s) for w in warehouse for s in stores]
x = LpVariable.dicts('num_of_shipments',index, lowBound=0, cat='Integer')

#Alternative2
#Q2 how to define in one line?
#x = LpVariable.dicts('num_of_shipments',[(w, s) for w in warehouse for s in stores], lowBound=0, cat='Integer')

# Define Objective
model3 += lpSum([costs[(w, s)] * x[(w, s)] for s in stores for w in warehouse])

# For each customer, sum warehouse shipments and set equal to customer demand
for s in stores:
    model2 += lpSum([x[(w, s)] for w in warehouse]) == demand[s]
    
# Solve Model
model3.solve()

1

In [43]:
#x[index[1]]
#costs[index[1]]

In [10]:
# Print status
print(LpStatus[model3.status])

# Print variables
for v in model3.variables():
    print(v.name, "=", v.varValue)

Optimal
num_of_shipments_('Istanbul',_'T100') = 0.0
num_of_shipments_('Istanbul',_'T101') = 0.0
num_of_shipments_('Istanbul',_'T102') = 0.0
num_of_shipments_('Istanbul',_'T103') = 0.0
num_of_shipments_('Konya',_'T100') = 0.0
num_of_shipments_('Konya',_'T101') = 0.0
num_of_shipments_('Konya',_'T102') = 0.0
num_of_shipments_('Konya',_'T103') = 0.0
